In [1]:
import ROOT
import numpy as np
import pandas as pd
import math
import os

Welcome to JupyROOT 6.18/00


In [2]:
!ls ../moreShapes_aug19/

vhbb_Wen.root  vhbb_Wmn.root  vhbb_Zee.root  vhbb_Zmm.root  vhbb_Znn.root


In [3]:
strd_dir = '../nominalShapesSTXS2017/standard_aug9/haddlevel3/'
extnd_dir = '../moreShapes_aug19/'
old_dir = '../adewit2017/'

In [4]:
f_test = ROOT.TFile("../moreShapes_aug19/vhbb_Znn.root")
f_test.ls()

TFile**		../moreShapes_aug19/vhbb_Znn.root	
 TFile*		../moreShapes_aug19/vhbb_Znn.root	
  KEY: TH1D	BDT_SR_high_Znn_QCD;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_TT_SL;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_TT_other;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_VVHF_WWLNu2Q;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_VVHF_ZZ;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_VVLF;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_WH_lep_PTV_0_75_hbb;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_WH_lep_PTV_150_250_0J_hbb;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_WH_lep_PTV_150_250_GE1J_hbb;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_WH_lep_PTV_75_150_hbb;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_WH_lep_PTV_GT250_hbb;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b_100to200;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b_200to400;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b_400to600;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b_600to800;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b_high;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b;1	nominal
  KEY: TH1D	BDT_SR_high_Znn_Wj0b_

In [5]:
_verbos = 1

In [6]:
def AsimovSignificance(h_s, h_b, sigma):
    n_bins = h_s.GetNbinsX()
    #sigma_b = sigma
    #ar_AS = pd.DataFrame()
    ar_AS = []
    for n in range(n_bins):
        #print n 
        s = h_s.GetBinContent(n)
        b = h_b.GetBinContent(n)
        sigma_b = h_b.GetBinError(n)
        #print s
        #print b
        
        AS = ROOT.RooStats.AsimovSignificance(s, b, sigma_b)

        AS_0 = ROOT.RooStats.AsimovSignificance(s, b, 0)
        #print AS
        ar_AS_ = np.array([s, b, AS, sigma_b, AS_0])
        ar_AS.append(ar_AS_)
    return ar_AS

In [7]:
def Sensitivity(df, name):
    if _verbos > 1:
        df.to_csv('test_.csv')
        !cat test_.csv
        #print(df)
    df['AsimovSignificance2%s' %(name)] = df['AsimovSignificance%s' % (name)]**2 
    sen = df[(df['AsimovSignificance2%s' %(name)] >0) & (df['AsimovSignificance2%s' %(name)] < float('inf'))].sum()
    sen = sen['AsimovSignificance2%s' %(name)]
    
    df['AsimovSignificance2%s_0' %(name)] = df['AsimovSignificance%s_0' % (name)]**2 
    sen_0 = df[(df['AsimovSignificance2%s_0' %(name)] >0) & (df['AsimovSignificance2%s_0' %(name)] < float('inf'))].sum()
    sen_0 = sen_0['AsimovSignificance2%s_0' %(name)]
    
    return math.sqrt(sen), math.sqrt(sen_0)

In [8]:
def calculate(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    if(_verbos>1): print("BDT_%s_%s_%s"%(reg, sam, pt));
                    #if('BDT_SR_med_Zee_0j_ZH_lep_PTV_0_75_hbb' == "BDT_%s_%s_%s"%(reg, sam, pt)): break;
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo','VVHF_WZLNu2Q', 'VVHF_ZZ2L2Q', 'VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            #print(sgma_b, sgma_b/math.sqrt(2.))
                            h_.SetBinError(n, sgma_b/math.sqrt(2.))
                            #print(sgma_b, h_.GetBinError(n))
                            if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_low.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "high" in region:
                #if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        if _verbos >1 : print(probe)
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(2.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_high.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(2.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(2.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med_ge1j.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

In [9]:
def calculate_4x(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    if(_verbos>1): print("BDT_%s_%s_%s"%(reg, sam, pt));
                    #if('BDT_SR_med_Zee_0j_ZH_lep_PTV_0_75_hbb' == "BDT_%s_%s_%s"%(reg, sam, pt)): break;
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo','VVHF_WZLNu2Q', 'VVHF_ZZ2L2Q', 'VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            #print(sgma_b, sgma_b/math.sqrt(2.))
                            h_.SetBinError(n, sgma_b/math.sqrt(4.))
                            #print(sgma_b, h_.GetBinError(n))
                            if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_low.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "high" in region:
                #if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        if _verbos >1 : print(probe)
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(4.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_high.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(4.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(4.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med_ge1j.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

In [10]:
def calculate_6x(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    if(_verbos>1): print("BDT_%s_%s_%s"%(reg, sam, pt));
                    #if('BDT_SR_med_Zee_0j_ZH_lep_PTV_0_75_hbb' == "BDT_%s_%s_%s"%(reg, sam, pt)): break;
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo','VVHF_WZLNu2Q', 'VVHF_ZZ2L2Q', 'VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            #print(sgma_b, sgma_b/math.sqrt(2.))
                            h_.SetBinError(n, sgma_b/math.sqrt(6.))
                            #print(sgma_b, h_.GetBinError(n))
                            if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_low.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "high" in region:
                #if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        if _verbos >1 : print(probe)
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(6.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_high.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(6.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(6.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med_ge1j.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

In [11]:
def calculate_8x(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    if(_verbos>1): print("BDT_%s_%s_%s"%(reg, sam, pt));
                    #if('BDT_SR_med_Zee_0j_ZH_lep_PTV_0_75_hbb' == "BDT_%s_%s_%s"%(reg, sam, pt)): break;
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo','VVHF_WZLNu2Q', 'VVHF_ZZ2L2Q', 'VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            #print(sgma_b, sgma_b/math.sqrt(2.))
                            h_.SetBinError(n, sgma_b/math.sqrt(8.))
                            #print(sgma_b, h_.GetBinError(n))
                            if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_low.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "high" in region:
                #if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        if _verbos >1 : print(probe)
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(8.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_high.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(8.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(8.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med_ge1j.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

In [12]:
def calculate_10x(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    if(_verbos>1): print("BDT_%s_%s_%s"%(reg, sam, pt));
                    #if('BDT_SR_med_Zee_0j_ZH_lep_PTV_0_75_hbb' == "BDT_%s_%s_%s"%(reg, sam, pt)): break;
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo','VVHF_WZLNu2Q', 'VVHF_ZZ2L2Q', 'VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            #print(sgma_b, sgma_b/math.sqrt(2.))
                            h_.SetBinError(n, sgma_b/math.sqrt(10.))
                            #print(sgma_b, h_.GetBinError(n))
                            if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_low.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "high" in region:
                #if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        if _verbos >1 : print(probe)
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(10.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_high.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(10.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, sgma_b/math.sqrt(10.))                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med_ge1j.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

In [13]:
def calculate_zero_stat(filename, channel, input_sigma, probe, n_, a, b):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", n_, a, b)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", n_, a, b)
    h_s_med_ge1j = ROOT.TH1F("h_s_med_ge1j", "h_s_med_ge1j", n_, a, b)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", n_, a, b)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg and "0j" in reg:
                    if(_verbos>1): print("BDT_%s_%s_%s"%(reg, sam, pt));
                    #if('BDT_SR_med_Zee_0j_ZH_lep_PTV_0_75_hbb' == "BDT_%s_%s_%s"%(reg, sam, pt)): break;
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "med" in reg and "ge1j" in reg:
                    #break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med_ge1j.Add(h_)                    
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee_0j', 'SR_med_Zee_ge1j', 'SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    sample_name = ['TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo', 'VVHF_WZLNu2Q', 'VVHF_ZZ2L2Q', 'VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
    h_b_low = ROOT.TH1F("h_b_low", "h_b_low", n_, a, b)
    h_b_med = ROOT.TH1F("h_b_med", "h_b_med", n_, a, b)
    h_b_med_ge1j = ROOT.TH1F("h_b_med_ge1j", "h_b_med_ge1j", n_, a, b)

    h_b_high = ROOT.TH1F("h_b_high", "h_b_high", n_, a, b)

    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            if "low" in region:
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            #print(sgma_b, sgma_b/math.sqrt(2.))
                            h_.SetBinError(n, 0.)
                            #print(sgma_b, h_.GetBinError(n))
                            if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_low.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "high" in region:
                #if channel=="Znn" and "Wj2b" in sample: continue 
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        if _verbos >1 : print(probe)
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, 0.)                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_high.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "0j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, 0.)                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            if "med" in region and "ge1j" in region:
                #break
                h_ = ROOT.TH1F()
                h_.Clear()
                try:
                    h_ = f_Zee.Get("BDT_%s_%s"%(region, sample))
                    if probe in sample:
                        n_bns = h_.GetNbinsX()
                        for n in range(n_bns):
                            sgma_b = h_.GetBinError(n)
                            h_.SetBinError(n, 0.)                
                    if _verbos > 1: print("BDT_%s_%s"%(region, sample))
                    h_b_med_ge1j.Add(h_)
                except:
                    print("ERROR: NOT EXIST:","BDT_%s_%s"%(region, sample))
            

    
    
    dict_AS = {}

    sigma_b = input_sigma
    region_names = [ 'SR_low_Zee', 'SR_low_Zmm','SR_med_Zee_0j', 'SR_med_Zee_ge1j','SR_med_Zmm_0j', 'SR_med_Zmm_ge1j', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn_0j', 'SR_med_Wmn_ge1j','SR_med_Wen_0j','SR_med_Wen_ge1j']

    for region in region_names:
        if "%s" %(channel) not in region: continue
        if _verbos > 1: print(region)
        #print("BDT_%s_TT" % (region))
        #print("BDT_%s_%s" % (region, sample))
        if "low" in region:
            h_s = h_s_low.Clone()
            h_b = h_b_low.Clone()
        if "high" in region:
            h_s = h_s_high.Clone()
            h_b = h_b_high.Clone()
        if "med" in region and "0j" in region:
            h_s = h_s_med.Clone()
            h_b = h_b_med.Clone()
        if "med" in region and "ge1j" in region:
            h_s = h_s_med_ge1j.Clone()
            h_b = h_b_med_ge1j.Clone()
        
                
        #h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
        l = AsimovSignificance(h_s, h_b, sigma_b)
        df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region), 'sigma_b', 'AsimovSignificance%s_0' %(region)])
        sen, sen_0 = Sensitivity(df, region)
        dict_AS.update({region : [sen, sen_0 ]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'AsimovSignificance_0'])

    return df_AS

Zee AMS:

In [34]:
_verbos=1
test_Zee = calculate(extnd_dir+"vhbb_Zee.root", "Zee", 0, "aaa", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_Zee.csv")
!cat test_Zee.csv

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

In [19]:
_verbos=1
probes = ['nominal','TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo', 'VVHF_ZZ2L2Q', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
for prb in probes:
    test_Zee = calculate(extnd_dir+"vhbb_Zee.root", "Zee", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_4x = calculate_4x(extnd_dir+"vhbb_Zee.root", "Zee", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_6x = calculate_6x(extnd_dir+"vhbb_Zee.root", "Zee", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_8x = calculate_8x(extnd_dir+"vhbb_Zee.root", "Zee", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_10x = calculate_10x(extnd_dir+"vhbb_Zee.root", "Zee", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_zero = calculate_zero_stat(extnd_dir+"vhbb_Zee.root", "Zee", 0, "%s"%(prb), 20, -1, 1)
    test_Zee['AMS_4x'] = test_Zee_4x['AsimovSignificance']
    test_Zee['AMS_6x'] = test_Zee_6x['AsimovSignificance']
    test_Zee['AMS_8x'] = test_Zee_8x['AsimovSignificance']
    test_Zee['AMS_10x'] = test_Zee_10x['AsimovSignificance']
    test_Zee['AMS_SampleZeroStatUnc'] = test_Zee_zero['AsimovSignificance']
    test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
    #print(test_Zee)
    test_Zee.to_csv("test_Zee%s.csv" %(prb))
    txt = "test_Zee%s.csv" %(prb)
    os.system("echo %s; cat test_Zee%s.csv" %(prb, prb) )

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

...

('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj2b_ZB100to20

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BD

('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zee_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zee_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Ze

In [20]:
_verbos=1
probes = ['nominal','TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo', 'VVHF_ZZ2L2Q',    'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
for prb in probes:
    test_Zee = calculate(extnd_dir+"vhbb_Zmm.root", "Zmm", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_4x = calculate_4x(extnd_dir+"vhbb_Zmm.root", "Zmm", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_6x = calculate_6x(extnd_dir+"vhbb_Zmm.root", "Zmm", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_8x = calculate_8x(extnd_dir+"vhbb_Zmm.root", "Zmm", 0, "%s"%(prb), 20, -1, 1)
    test_Zee_10x = calculate_10x(extnd_dir+"vhbb_Zmm.root", "Zmm", 0, "%s"%(prb), 20, -1, 1)
    test_Zee['AMS_4x'] = test_Zee_4x['AsimovSignificance']
    test_Zee['AMS_6x'] = test_Zee_6x['AsimovSignificance']
    test_Zee['AMS_8x'] = test_Zee_8x['AsimovSignificance']
    test_Zee['AMS_10x'] = test_Zee_10x['AsimovSignificance']
    test_Zee_zero = calculate_zero_stat(extnd_dir+"vhbb_Zmm.root", "Zmm", 0, "%s"%(prb), 20, -1, 1)
    test_Zee['AMS_SampleZeroStatUnc'] = test_Zee_zero['AsimovSignificance']
    test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
    #print(test_Zee)
    test_Zee.to_csv("test_Zmm%s.csv" %(prb))
    txt = "test_Zmm%s.csv" %(prb)
    os.system("echo %s; cat test_Zmm%s.csv" %(prb, prb) )

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

...

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Zmm_VVHF_WWLNu2Q')
('ERRO

('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_low_Zmm_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zmm_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Zm

nominal
,AsimovSignificance,AsimovSignificance_0,AMS_4x,AMS_6x,AMS_8x,AMS_10x,AMS_SampleZeroStatUnc,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.522684009158112,0.5754850262845952,0.522684009158112,0.522684009158112,0.522684009158112,0.522684009158112,0.522684009158112,0.09175046215776164
SR_med_Zmm_ge1j,0.4255087355097661,0.473659235341788,0.4255087355097661,0.4255087355097661,0.4255087355097661,0.4255087355097661,0.4255087355097661,0.10165641507502111
SR_med_Zmm_0j,0.6752993242269925,0.7312765992101066,0.6752993242269925,0.6752993242269925,0.6752993242269925,0.6752993242269925,0.6752993242269925,0.07654733522661378
SR_low_Zmm,0.5565597941104495,0.6876238203187913,0.5565597941104495,0.5565597941104495,0.5565597941104495,0.5565597941104495,0.5565597941104495,0.19060425531445202
TT_SL
,AsimovSignificance,AsimovSignificance_0,AMS_4x,AMS_6x,AMS_8x,AMS_10x,AMS_SampleZeroStatUnc,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.5228003320323801,0.5754850262845952,0.5228588874850149,0.522878464791121,0.5228882645119

In [21]:
_verbos=1
probes = ['nominal','TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo', 'VVHF_ZZ2L2Q', 'VVHF_ZZ2L2Q',  'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
for prb in probes:
    test_Zee = calculate(extnd_dir+"vhbb_Wen.root", "Wen", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_4x = calculate_4x(extnd_dir+"vhbb_Wen.root", "Wen", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_6x = calculate_6x(extnd_dir+"vhbb_Wen.root", "Wen", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_8x = calculate_8x(extnd_dir+"vhbb_Wen.root", "Wen", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_10x = calculate_10x(extnd_dir+"vhbb_Wen.root", "Wen", 0, "%s"%(prb), 15, 0, 1)
    test_Zee['AMS_4x'] = test_Zee_4x['AsimovSignificance']
    test_Zee['AMS_6x'] = test_Zee_6x['AsimovSignificance']
    test_Zee['AMS_8x'] = test_Zee_8x['AsimovSignificance']
    test_Zee['AMS_10x'] = test_Zee_10x['AsimovSignificance']
    test_Zee_zero = calculate_zero_stat(extnd_dir+"vhbb_Wen.root", "Wen", 0, "%s"%(prb), 15, 0, 1)
    test_Zee['AMS_SampleZeroStatUnc'] = test_Zee_zero['AsimovSignificance']
    test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
    #print(test_Zee)
    test_Zee.to_csv("test_Wen%s.csv" %(prb))
    txt = "test_Wen%s.csv" %(prb)
    os.system("echo %s; cat test_Wen%s.csv" %(prb, prb) )

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:',

...

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERRO

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wen_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wen_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

nominal
,AsimovSignificance,AsimovSignificance_0,AMS_4x,AMS_6x,AMS_8x,AMS_10x,AMS_SampleZeroStatUnc,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.28068247139573976,0.3092701849186053,0.28068247139573976,0.28068247139573976,0.28068247139573976,0.28068247139573976,0.28068247139573976,0.09243604756271397
SR_med_Wen_0j,0.9091405567448573,0.9964726810703879,0.9091405567448573,0.9091405567448573,0.9091405567448573,0.9091405567448573,0.9091405567448573,0.08764126301156643
SR_high_Wen,0.8704705446404883,0.9338817228345869,0.8704705446404883,0.8704705446404883,0.8704705446404883,0.8704705446404883,0.8704705446404883,0.06790065234559713
TT_SL
,AsimovSignificance,AsimovSignificance_0,AMS_4x,AMS_6x,AMS_8x,AMS_10x,AMS_SampleZeroStatUnc,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2858550908041021,0.3092701849186053,0.2885615969049448,0.2894826585045564,0.2899468297804691,0.2902265104457758,0.2913541775720077,0.075710803227494
SR_med_Wen_0j,0.9164819826327881,0.9964726810703879,0.9202454632993996,0.9215141590163

In [22]:
_verbos=1
probes = ['nominal','TT_SL', 'TT_other', 'VVHF_WWLNu2Qnlo', 'VVHF_ZZ2L2Q', 'VVHF_ZZ2L2Q',  'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
for prb in probes:
    test_Zee = calculate(extnd_dir+"vhbb_Wmn.root", "Wmn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_4x = calculate_4x(extnd_dir+"vhbb_Wmn.root", "Wmn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_6x = calculate_6x(extnd_dir+"vhbb_Wmn.root", "Wmn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_8x = calculate_8x(extnd_dir+"vhbb_Wmn.root", "Wmn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_10x = calculate_10x(extnd_dir+"vhbb_Wmn.root", "Wmn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee['AMS_4x'] = test_Zee_4x['AsimovSignificance']
    test_Zee['AMS_6x'] = test_Zee_6x['AsimovSignificance']
    test_Zee['AMS_8x'] = test_Zee_8x['AsimovSignificance']
    test_Zee['AMS_10x'] = test_Zee_10x['AsimovSignificance']
    test_Zee_zero = calculate_zero_stat(extnd_dir+"vhbb_Wmn.root", "Wmn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee['AMS_SampleZeroStatUnc'] = test_Zee_zero['AsimovSignificance']
    test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
    #print(test_Zee)
    test_Zee.to_csv("test_Wmn%s.csv" %(prb))
    txt = "test_Wmn%s.csv" %(prb)
    os.system("echo %s; cat test_Wmn%s.csv" %(prb, prb) )

('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERRO

('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT

('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB2

('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

...

('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:'

('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf'

('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_high_Wmn_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_0j_Zj2b_ZB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_WWLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Wmn_ge1j_VVHF_ZZ')
('ERROR: NOT EXIST:', 'BDT_SR

In [23]:
_verbos=1
probes = ['nominal','TT_SL', 'TT_other','VVHF_WWLNu2Q', 'VVHF_ZZ', 'VVLF',
'Wj0b_high','Wj0b_low', 'Wj0b_100to200', 'Wj0b_200to400', 'Wj0b_400to600', 'Wj0b_600to800',
'Wj1b_high', 'Wj1b_low', 'Wj1b_100to200', 'Wj1b_200to400', 'Wj1b_400to600', 'Wj1b_600to800', 'Wj1b_BGen100to200', 'Wj1b_BGen200toInf', 'Wj1b_WB100to200', 'Wj1b_WB200toInf',
'Wj2b_high', 'Wj2b_low','Wj2b_100to200','Wj2b_200to400', 'Wj2b_400to600', 'Wj2b_600to800', 'Wj2b_BGen100to200', 'Wj2b_BGen200toInf', 'Wj2b_WB100to200', 'Wj2b_WB200toInf', 
'Zj0b_high', 'Zj0b_low', 'Zj0b_200to400',
'Zj1b_high', 'Zj1b_low', 'Zj1b_200to400', 'Zj1b_BGen100to200', 'Zj1b_BGen200toInf', 'Zj1b_DYB100to200', 'Zj1b_DYB200toInf','Zj1b_ZB100to200', 'Zj1b_ZB200toInf'
'Zj2b_high', 'Zj2b_low', 'Zj2b_200to400', 'Zj2b_BGen100to200', 'Zj2b_BGen200toInf', 'Zj2b_DYB100to200', 'Zj2b_DYB200toInf','Zj2b_ZB100to200', 'Zj2b_ZB200toInf', 's_Top_tW', 's_Top_schan', 's_Top_tchan']
for prb in probes:
    test_Zee = calculate(extnd_dir+"vhbb_Znn.root", "Znn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_zero = calculate_zero_stat(extnd_dir+"vhbb_Znn.root", "Znn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_4x = calculate_4x(extnd_dir+"vhbb_Znn.root", "Znn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_6x = calculate_6x(extnd_dir+"vhbb_Znn.root", "Znn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_8x = calculate_8x(extnd_dir+"vhbb_Znn.root", "Znn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee_10x = calculate_10x(extnd_dir+"vhbb_Znn.root", "Znn", 0, "%s"%(prb), 15, 0, 1)
    test_Zee['AMS_4x'] = test_Zee_4x['AsimovSignificance']
    test_Zee['AMS_6x'] = test_Zee_6x['AsimovSignificance']
    test_Zee['AMS_8x'] = test_Zee_8x['AsimovSignificance']
    test_Zee['AMS_10x'] = test_Zee_10x['AsimovSignificance']
    test_Zee['AMS_SampleZeroStatUnc'] = test_Zee_zero['AsimovSignificance']
    test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
    #print(test_Zee)
    test_Zee.to_csv("test_Znn%s.csv" %(prb))
    txt = "test_Znn%s.csv" %(prb)
    os.system("echo %s; cat test_Znn%s.csv" %(prb, prb) )

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

...

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_0j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WWLNu2Qnlo')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_WZLNu2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_VVHF_ZZ2L2Q')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj1b_ZB200toInfZj2b_high')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB100to200')
('ERROR: NOT EXIST:', 'BDT_SR_med_Znn_ge1j_Zj2b_DYB200toInf')
('ERROR: NOT EXIST:', 'BDT_S

nominal
,AsimovSignificance,AsimovSignificance_0,AMS_4x,AMS_6x,AMS_8x,AMS_10x,AMS_SampleZeroStatUnc,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.14948873413647862,0.1593156972671754,0.14948873413647862,0.14948873413647862,0.14948873413647862,0.14948873413647862,0.14948873413647862,0.06168232822793839
SR_med_Znn_0j,0.26156603986565086,0.27787521838151225,0.26156603986565086,0.26156603986565086,0.26156603986565086,0.26156603986565086,0.26156603986565086,0.05869245415569768
SR_high_Znn,0.19571682413367272,0.20777024336796854,0.19571682413367272,0.19571682413367272,0.19571682413367272,0.19571682413367272,0.19571682413367272,0.058013212281552644
TT_SL
,AsimovSignificance,AsimovSignificance_0,AMS_4x,AMS_6x,AMS_8x,AMS_10x,AMS_SampleZeroStatUnc,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.15155750174298033,0.1593156972671754,0.15263033520427605,0.15299392385488902,0.1531768640901524,0.1532869982181456,0.1537303377170424,0.04869699381338697
SR_med_Znn_0j,0.26349202511936176,0.27787521838151225,0.2644753046

In [22]:
test_Zee = calculate(strd_dir+"vhbb_Zee.root", "Zee", 0, "VV", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeVV.csv")
!cat test_ZeeVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.05632483044303189,0.0603604769949326,0.0668590898020821
SR_med_Zee_ge1j,0.09194759712699825,0.0981589237855223,0.06327826772118877
SR_med_Zee_0j,0.08468588595610756,0.0906162665921985,0.06544498972551482
SR_low_Zee,0.05881641025603157,0.06899150690703239,0.14748332232707992


In [18]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "VVLF", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeVVLF.csv")
!cat test_ZeeVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6323917039037503,0.6789413012018976,0.06856203506215144
SR_med_Zee_ge1j,0.5555201904389752,0.5920300987482715,0.06166900700908484
SR_med_Zee_0j,0.79998045197544,0.8584333317152688,0.06809250943580192
SR_low_Zee,0.5900190317348472,0.7082241224187588,0.1669035082852195


In [19]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 0, "j", 20, -1, 1)
test_Zee['ratio=(AS_0-AS)/AS_0'] = (test_Zee['AsimovSignificance_0'] - test_Zee['AsimovSignificance'])/test_Zee['AsimovSignificance_0']
#print(test_Zee)
test_Zee.to_csv("test_ZeeVj.csv")
!cat test_ZeeVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zee,0.6498118119071743,0.6789413012018976,0.04290428236308014
SR_med_Zee_ge1j,0.5683976865339366,0.5920300987482715,0.03991758571785599
SR_med_Zee_0j,0.8232523690396396,0.8584333317152688,0.040982754718217614
SR_low_Zee,0.6330414029329852,0.7082241224187588,0.10615667711092105


Zmm AMS:

In [39]:
test_Zmm = calculate(strd_dir+"vhbb_Zmm.root", "Zmm", 0, "aaa", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_Zmm.csv")
!cat test_Zmm.csv

BDT_SR_low_Zmm_TT
BDT_SR_low_Zmm_VVHF
BDT_SR_low_Zmm_VVLF
BDT_SR_low_Zmm_Wj0b
BDT_SR_low_Zmm_Wj1b
BDT_SR_low_Zmm_Wj2b
BDT_SR_low_Zmm_Zj0b
BDT_SR_low_Zmm_Zj1b
BDT_SR_low_Zmm_Zj2b
BDT_SR_low_Zmm_s_Top
BDT_SR_med_Zmm_0j_TT
BDT_SR_med_Zmm_0j_VVHF
BDT_SR_med_Zmm_0j_VVLF
BDT_SR_med_Zmm_0j_Wj0b
BDT_SR_med_Zmm_0j_Wj1b
BDT_SR_med_Zmm_0j_Wj2b
BDT_SR_med_Zmm_0j_Zj0b
BDT_SR_med_Zmm_0j_Zj1b
BDT_SR_med_Zmm_0j_Zj2b
BDT_SR_med_Zmm_0j_s_Top
BDT_SR_med_Zmm_ge1j_TT
BDT_SR_med_Zmm_ge1j_VVHF
BDT_SR_med_Zmm_ge1j_VVLF
BDT_SR_med_Zmm_ge1j_Wj0b
BDT_SR_med_Zmm_ge1j_Wj1b
BDT_SR_med_Zmm_ge1j_Wj2b
BDT_SR_med_Zmm_ge1j_Zj0b
BDT_SR_med_Zmm_ge1j_Zj1b
BDT_SR_med_Zmm_ge1j_Zj2b
BDT_SR_med_Zmm_ge1j_s_Top
BDT_SR_high_Zmm_TT
BDT_SR_high_Zmm_VVHF
BDT_SR_high_Zmm_VVLF
BDT_SR_high_Zmm_Wj0b
BDT_SR_high_Zmm_Wj1b
BDT_SR_high_Zmm_Wj2b
BDT_SR_high_Zmm_Zj0b
BDT_SR_high_Zmm_Zj1b
BDT_SR_high_Zmm_Zj2b
BDT_SR_high_Zmm_s_Top
SR_low_Zmm
,signal,background,AsimovSignificanceSR_low_Zmm,sigma_b,AsimovSignificanceSR_low_Zmm_0
0,0.0,0.0,,0.0,


In [40]:
test_Zmm = calculate(old_dir+"vhbb_Zmm-2017.root", "Zmm", 0, "aaa", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_Zmm.csv")
!cat test_Zmm.csv

BDT_SR_low_Zmm_TT
BDT_SR_low_Zmm_VVHF
BDT_SR_low_Zmm_VVLF
BDT_SR_low_Zmm_Wj0b
BDT_SR_low_Zmm_Wj1b
BDT_SR_low_Zmm_Wj2b
BDT_SR_low_Zmm_Zj0b
BDT_SR_low_Zmm_Zj1b
BDT_SR_low_Zmm_Zj2b
BDT_SR_low_Zmm_s_Top
BDT_SR_med_Zmm_0j_TT
BDT_SR_med_Zmm_0j_VVHF
BDT_SR_med_Zmm_0j_VVLF
BDT_SR_med_Zmm_0j_Wj0b
BDT_SR_med_Zmm_0j_Wj1b
BDT_SR_med_Zmm_0j_Wj2b
BDT_SR_med_Zmm_0j_Zj0b
BDT_SR_med_Zmm_0j_Zj1b
BDT_SR_med_Zmm_0j_Zj2b
BDT_SR_med_Zmm_0j_s_Top
BDT_SR_med_Zmm_ge1j_TT
BDT_SR_med_Zmm_ge1j_VVHF
BDT_SR_med_Zmm_ge1j_VVLF
BDT_SR_med_Zmm_ge1j_Wj0b
BDT_SR_med_Zmm_ge1j_Wj1b
BDT_SR_med_Zmm_ge1j_Wj2b
BDT_SR_med_Zmm_ge1j_Zj0b
BDT_SR_med_Zmm_ge1j_Zj1b
BDT_SR_med_Zmm_ge1j_Zj2b
BDT_SR_med_Zmm_ge1j_s_Top
BDT_SR_high_Zmm_TT
BDT_SR_high_Zmm_VVHF
BDT_SR_high_Zmm_VVLF
BDT_SR_high_Zmm_Wj0b
BDT_SR_high_Zmm_Wj1b
BDT_SR_high_Zmm_Wj2b
BDT_SR_high_Zmm_Zj0b
BDT_SR_high_Zmm_Zj1b
BDT_SR_high_Zmm_Zj2b
BDT_SR_high_Zmm_s_Top
SR_low_Zmm
,signal,background,AsimovSignificanceSR_low_Zmm,sigma_b,AsimovSignificanceSR_low_Zmm_0
0,0.0,0.0,,0.0,


In [21]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "TT", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmTT.csv")
!cat test_ZmmTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7237553686818818,0.7772729317231171,0.06885298697150494
SR_med_Zmm_ge1j,0.6467140486359787,0.6950647061668356,0.06956281494639914
SR_med_Zmm_0j,0.9243318386196564,0.987916516848541,0.06436240020737789
SR_low_Zmm,0.7281239055293004,0.868058059138925,0.16120368002623356


In [22]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "VV", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmVV.csv")
!cat test_ZmmVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7281129761204621,0.7772729317231171,0.06324670987020412
SR_med_Zmm_ge1j,0.64787847384911,0.6950647061668356,0.06788753895727156
SR_med_Zmm_0j,0.9253234564750549,0.987916516848541,0.06335865359672121
SR_low_Zmm,0.7253216922697894,0.868058059138925,0.1644318203908201


In [23]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "VVLF", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmVVLF.csv")
!cat test_ZmmVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7239820808076615,0.7772729317231171,0.06856131062909451
SR_med_Zmm_ge1j,0.6449237753384364,0.6950647061668356,0.07213850794542277
SR_med_Zmm_0j,0.9228400254142445,0.987916516848541,0.06587246019723494
SR_low_Zmm,0.7242735037814846,0.868058059138925,0.16563933004673476


In [24]:
test_Zmm = calculate("vhbb_Zmm-2017.root", "Zmm", 0, "j", 20, -1, 1)
test_Zmm['ratio=(AS_0-AS)/AS_0'] = (test_Zmm['AsimovSignificance_0'] - test_Zmm['AsimovSignificance'])/test_Zmm['AsimovSignificance_0']
#print(test_Zee)
test_Zmm.to_csv("test_ZmmVj.csv")
!cat test_ZmmVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_high_Zmm,0.7439048525701141,0.7772729317231171,0.04292968118551373
SR_med_Zmm_ge1j,0.6630084790227099,0.6950647061668356,0.04611977397170744
SR_med_Zmm_0j,0.9488391393241543,0.987916516848541,0.03955534385541373
SR_low_Zmm,0.7785468307058333,0.868058059138925,0.10311663775333511


Wen AMS:

In [41]:
test_Wen = calculate(strd_dir+"vhbb_Wen.root", "Wen", 0, "aaa", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_Wen.csv")
!cat test_Wen.csv

BDT_SR_high_Wen_TT
BDT_SR_high_Wen_VVHF
BDT_SR_high_Wen_VVLF
BDT_SR_high_Wen_Wj0b
BDT_SR_high_Wen_Wj1b
BDT_SR_high_Wen_Wj2b
BDT_SR_high_Wen_Zj0b
BDT_SR_high_Wen_Zj1b
BDT_SR_high_Wen_Zj2b
BDT_SR_high_Wen_s_Top
BDT_SR_med_Wen_0j_TT
BDT_SR_med_Wen_0j_VVHF
BDT_SR_med_Wen_0j_VVLF
BDT_SR_med_Wen_0j_Wj0b
BDT_SR_med_Wen_0j_Wj1b
BDT_SR_med_Wen_0j_Wj2b
BDT_SR_med_Wen_0j_Zj0b
BDT_SR_med_Wen_0j_Zj1b
BDT_SR_med_Wen_0j_Zj2b
BDT_SR_med_Wen_0j_s_Top
BDT_SR_med_Wen_ge1j_TT
BDT_SR_med_Wen_ge1j_VVHF
BDT_SR_med_Wen_ge1j_VVLF
BDT_SR_med_Wen_ge1j_Wj0b
BDT_SR_med_Wen_ge1j_Wj1b
BDT_SR_med_Wen_ge1j_Wj2b
BDT_SR_med_Wen_ge1j_Zj0b
BDT_SR_med_Wen_ge1j_Zj1b
BDT_SR_med_Wen_ge1j_Zj2b
BDT_SR_med_Wen_ge1j_s_Top
SR_high_Wen
,signal,background,AsimovSignificanceSR_high_Wen,sigma_b,AsimovSignificanceSR_high_Wen_0
0,0.0,0.0,,0.0,
1,0.0,20.47711944580078,0.0,1.8427343039073434,0.0
2,0.0006721220561303198,140.2471466064453,5.130118636376204e-05,5.603124402578536,5.675449007056043e-05
3,0.0011239233426749706,205.7563629150390

In [42]:
test_Wen = calculate(old_dir+"vhbb_Wen-2017.root", "Wen", 0, "aaa", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_Wen.csv")
!cat test_Wen.csv

BDT_SR_high_Wen_TT
BDT_SR_high_Wen_VVHF
BDT_SR_high_Wen_VVLF
BDT_SR_high_Wen_Wj0b
BDT_SR_high_Wen_Wj1b
BDT_SR_high_Wen_Wj2b
BDT_SR_high_Wen_Zj0b
BDT_SR_high_Wen_Zj1b
BDT_SR_high_Wen_Zj2b
BDT_SR_high_Wen_s_Top
BDT_SR_med_Wen_0j_TT
BDT_SR_med_Wen_0j_VVHF
BDT_SR_med_Wen_0j_VVLF
BDT_SR_med_Wen_0j_Wj0b
BDT_SR_med_Wen_0j_Wj1b
BDT_SR_med_Wen_0j_Wj2b
BDT_SR_med_Wen_0j_Zj0b
BDT_SR_med_Wen_0j_Zj1b
BDT_SR_med_Wen_0j_Zj2b
BDT_SR_med_Wen_0j_s_Top
BDT_SR_med_Wen_ge1j_TT
BDT_SR_med_Wen_ge1j_VVHF
BDT_SR_med_Wen_ge1j_VVLF
BDT_SR_med_Wen_ge1j_Wj0b
BDT_SR_med_Wen_ge1j_Wj1b
BDT_SR_med_Wen_ge1j_Wj2b
BDT_SR_med_Wen_ge1j_Zj0b
BDT_SR_med_Wen_ge1j_Zj1b
BDT_SR_med_Wen_ge1j_Zj2b
BDT_SR_med_Wen_ge1j_s_Top
SR_high_Wen
,signal,background,AsimovSignificanceSR_high_Wen,sigma_b,AsimovSignificanceSR_high_Wen_0
0,0.0,0.0,,0.0,
1,0.005397445522248745,20.096298217773438,0.0010707610645317993,2.3043817786320684,0.0012039564981758772
2,0.09036188572645187,145.73501586914062,0.006493216954651262,6.919530176025685,0.007484425

In [26]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "TT", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenTT.csv")
!cat test_WenTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2740785457944522,0.2956839751666351,0.07306932802160479
SR_med_Wen_0j,0.9351361559321311,1.0301115529789004,0.09219913782358555
SR_high_Wen,0.9626951663067066,1.0324724111996086,0.0675826725595789


In [27]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "VV", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenVV.csv")
!cat test_WenVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2635669298208357,0.2956839751666351,0.10861949934114482
SR_med_Wen_0j,0.9159688620431478,1.0301115529789004,0.11080614580592967
SR_high_Wen,0.9489635197147108,1.0324724111996086,0.08088244352008452


In [28]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "VVLF", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenVVLF.csv")
!cat test_WenVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.26355388405486696,0.2956839751666351,0.10866361998028791
SR_med_Wen_0j,0.915905168850569,1.0301115529789004,0.11086797716040242
SR_high_Wen,0.9488430560651311,1.0324724111996086,0.08099911845325757


In [29]:
test_Wen = calculate("vhbb_Wen-2017.root", "Wen", 0, "j", 15, 0, 1)
test_Wen['ratio=(AS_0-AS)/AS_0'] = (test_Wen['AsimovSignificance_0'] - test_Wen['AsimovSignificance'])/test_Wen['AsimovSignificance_0']
#print(test_Wen)
test_Wen.to_csv("test_WenVj.csv")
!cat test_WenVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wen_ge1j,0.2654193539326949,0.2956839751666351,0.10235462106759191
SR_med_Wen_0j,0.9316259790709344,1.0301115529789004,0.09560670747082024
SR_high_Wen,0.9642207924394437,1.0324724111996086,0.066105029073721


Wmn AMS:

In [44]:
test_Wmn = calculate(old_dir+"vhbb_Wmn-2017.root", "Wmn", 0, "aaa", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_Wmn.csv")
!cat test_Wmn.csv

BDT_SR_high_Wmn_TT
BDT_SR_high_Wmn_VVHF
BDT_SR_high_Wmn_VVLF
BDT_SR_high_Wmn_Wj0b
BDT_SR_high_Wmn_Wj1b
BDT_SR_high_Wmn_Wj2b
BDT_SR_high_Wmn_Zj0b
BDT_SR_high_Wmn_Zj1b
BDT_SR_high_Wmn_Zj2b
BDT_SR_high_Wmn_s_Top
BDT_SR_med_Wmn_0j_TT
BDT_SR_med_Wmn_0j_VVHF
BDT_SR_med_Wmn_0j_VVLF
BDT_SR_med_Wmn_0j_Wj0b
BDT_SR_med_Wmn_0j_Wj1b
BDT_SR_med_Wmn_0j_Wj2b
BDT_SR_med_Wmn_0j_Zj0b
BDT_SR_med_Wmn_0j_Zj1b
BDT_SR_med_Wmn_0j_Zj2b
BDT_SR_med_Wmn_0j_s_Top
BDT_SR_med_Wmn_ge1j_TT
BDT_SR_med_Wmn_ge1j_VVHF
BDT_SR_med_Wmn_ge1j_VVLF
BDT_SR_med_Wmn_ge1j_Wj0b
BDT_SR_med_Wmn_ge1j_Wj1b
BDT_SR_med_Wmn_ge1j_Wj2b
BDT_SR_med_Wmn_ge1j_Zj0b
BDT_SR_med_Wmn_ge1j_Zj1b
BDT_SR_med_Wmn_ge1j_Zj2b
BDT_SR_med_Wmn_ge1j_s_Top
SR_high_Wmn
,signal,background,AsimovSignificanceSR_high_Wmn,sigma_b,AsimovSignificanceSR_high_Wmn_0
0,0.0,0.0,,0.0,
1,0.00978492945432663,46.90256881713867,0.001196034863701854,4.4746449812716,0.0014287106614533649
2,0.10739629715681076,199.73861694335938,0.006466575533940466,8.719000729145053,0.007598351442920

In [43]:
test_Wmn = calculate(strd_dir+"vhbb_Wmn.root", "Wmn", 0, "aaa", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_Wmn.csv")
!cat test_Wmn.csv

BDT_SR_high_Wmn_TT
BDT_SR_high_Wmn_VVHF
BDT_SR_high_Wmn_VVLF
BDT_SR_high_Wmn_Wj0b
BDT_SR_high_Wmn_Wj1b
BDT_SR_high_Wmn_Wj2b
BDT_SR_high_Wmn_Zj0b
BDT_SR_high_Wmn_Zj1b
BDT_SR_high_Wmn_Zj2b
BDT_SR_high_Wmn_s_Top
BDT_SR_med_Wmn_0j_TT
BDT_SR_med_Wmn_0j_VVHF
BDT_SR_med_Wmn_0j_VVLF
BDT_SR_med_Wmn_0j_Wj0b
BDT_SR_med_Wmn_0j_Wj1b
BDT_SR_med_Wmn_0j_Wj2b
BDT_SR_med_Wmn_0j_Zj0b
BDT_SR_med_Wmn_0j_Zj1b
BDT_SR_med_Wmn_0j_Zj2b
BDT_SR_med_Wmn_0j_s_Top
BDT_SR_med_Wmn_ge1j_TT
BDT_SR_med_Wmn_ge1j_VVHF
BDT_SR_med_Wmn_ge1j_VVLF
BDT_SR_med_Wmn_ge1j_Wj0b
BDT_SR_med_Wmn_ge1j_Wj1b
BDT_SR_med_Wmn_ge1j_Wj2b
BDT_SR_med_Wmn_ge1j_Zj0b
BDT_SR_med_Wmn_ge1j_Zj1b
BDT_SR_med_Wmn_ge1j_Zj2b
BDT_SR_med_Wmn_ge1j_s_Top
SR_high_Wmn
,signal,background,AsimovSignificanceSR_high_Wmn,sigma_b,AsimovSignificanceSR_high_Wmn_0
0,0.0,0.0,,0.0,
1,0.00020894901535939425,50.90542984008789,2.633775953339747e-05,3.468423379358958,2.9285853623860577e-05
2,0.00046845205361023545,215.8934783935547,2.092233196767674e-05,16.894075205337916,3.1882

In [31]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "TT", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnTT.csv")
!cat test_WmnTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.3343310600362577,0.3613190740413683,0.07469302326956787
SR_med_Wmn_0j,1.1986095949661406,1.2944830714163442,0.07406313652700357
SR_high_Wmn,1.0226111061025867,1.1346675321666178,0.09875705692403335


In [32]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "VV", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnVV.csv")
!cat test_WmnVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.32183715107775795,0.3613190740413683,0.10927162665951584
SR_med_Wmn_0j,1.1769750993991746,1.2944830714163442,0.09077598202083835
SR_high_Wmn,1.0045000508858248,1.1346675321666178,0.11471860927600663


In [33]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "VVLF", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnVVLF.csv")
!cat test_WmnVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.3218261141555109,0.3613190740413683,0.10930217285272847
SR_med_Wmn_0j,1.1762728611133275,1.2944830714163442,0.0913184675128106
SR_high_Wmn,1.004128240107395,1.1346675321666178,0.1150462918507604


In [34]:
test_Wmn = calculate("vhbb_Wmn-2017.root", "Wmn", 0, "j", 15, 0, 1)
test_Wmn['ratio=(AS_0-AS)/AS_0'] = (test_Wmn['AsimovSignificance_0'] - test_Wmn['AsimovSignificance'])/test_Wmn['AsimovSignificance_0']
#print(test_Wen)
test_Wmn.to_csv("test_WmnVj.csv")
!cat test_WmnVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Wmn_ge1j,0.32451300398297617,0.3613190740413683,0.1018658374348046
SR_med_Wmn_0j,1.1932983943253646,1.2944830714163442,0.07816608754896237
SR_high_Wmn,1.0340211898723852,1.1346675321666178,0.08870117408052647


Znn AMS:

In [46]:
_verbos = 2
test_Znn = calculate(strd_dir+"vhbb_Znn.root", "Znn", 0,"aaa", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_Znn.csv")
!cat test_Znn.csv

BDT_SR_med_Znn_0j_TT
BDT_SR_med_Znn_0j_VVHF
BDT_SR_med_Znn_0j_VVLF
BDT_SR_med_Znn_0j_Wj0b
BDT_SR_med_Znn_0j_Wj1b
BDT_SR_med_Znn_0j_Wj2b
BDT_SR_med_Znn_0j_Zj0b
BDT_SR_med_Znn_0j_Zj1b
BDT_SR_med_Znn_0j_Zj2b
BDT_SR_med_Znn_0j_s_Top
BDT_SR_med_Znn_ge1j_TT
BDT_SR_med_Znn_ge1j_VVHF
BDT_SR_med_Znn_ge1j_VVLF
BDT_SR_med_Znn_ge1j_Wj0b
BDT_SR_med_Znn_ge1j_Wj1b
BDT_SR_med_Znn_ge1j_Wj2b
BDT_SR_med_Znn_ge1j_Zj0b
BDT_SR_med_Znn_ge1j_Zj1b
BDT_SR_med_Znn_ge1j_Zj2b
BDT_SR_med_Znn_ge1j_s_Top
BDT_SR_high_Znn_TT
BDT_SR_high_Znn_VVHF
BDT_SR_high_Znn_VVLF
BDT_SR_high_Znn_Wj0b
BDT_SR_high_Znn_Wj1b
BDT_SR_high_Znn_Zj0b
BDT_SR_high_Znn_Zj1b
BDT_SR_high_Znn_Zj2b
BDT_SR_high_Znn_s_Top
SR_med_Znn_0j
,signal,background,AsimovSignificanceSR_med_Znn_0j,sigma_b,AsimovSignificanceSR_med_Znn_0j_0
0,0.0,0.0,,0.0,
1,0.0,4.955682277679443,0.0,0.9526593433899314,0.0
2,9.546062938170508e-05,114.81249237060547,7.805091531023575e-06,5.906676703929802,8.91039376386019e-06
3,0.0004022744542453438,690.287841796875,6.7274526409872

In [47]:
_verbos = 2
test_Znn = calculate(old_dir+"vhbb_Znn-2017.root", "Znn", 0,"aaa", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_Znn.csv")
!cat test_Znn.csv

BDT_SR_med_Znn_0j_TT
BDT_SR_med_Znn_0j_VVHF
BDT_SR_med_Znn_0j_VVLF
BDT_SR_med_Znn_0j_Wj0b
BDT_SR_med_Znn_0j_Wj1b
BDT_SR_med_Znn_0j_Wj2b
BDT_SR_med_Znn_0j_Zj0b
BDT_SR_med_Znn_0j_Zj1b
BDT_SR_med_Znn_0j_Zj2b
BDT_SR_med_Znn_0j_s_Top
BDT_SR_med_Znn_ge1j_TT
BDT_SR_med_Znn_ge1j_VVHF
BDT_SR_med_Znn_ge1j_VVLF
BDT_SR_med_Znn_ge1j_Wj0b
BDT_SR_med_Znn_ge1j_Wj1b
BDT_SR_med_Znn_ge1j_Wj2b
BDT_SR_med_Znn_ge1j_Zj0b
BDT_SR_med_Znn_ge1j_Zj1b
BDT_SR_med_Znn_ge1j_Zj2b
BDT_SR_med_Znn_ge1j_s_Top
BDT_SR_high_Znn_TT
BDT_SR_high_Znn_VVHF
BDT_SR_high_Znn_VVLF
BDT_SR_high_Znn_Wj0b
BDT_SR_high_Znn_Wj1b
BDT_SR_high_Znn_Zj0b
BDT_SR_high_Znn_Zj1b
BDT_SR_high_Znn_Zj2b
BDT_SR_high_Znn_s_Top
SR_med_Znn_0j
,signal,background,AsimovSignificanceSR_med_Znn_0j,sigma_b,AsimovSignificanceSR_med_Znn_0j_0
0,0.0,0.0,,0.0,
1,0.0,1.471318244934082,0.0,0.6891327011404029,0.0
2,0.020473472774028778,89.1115951538086,0.0017019368960675177,7.4553511503824295,0.002168741436197696
3,0.12297345697879791,462.9914245605469,0.0049216446179575

In [36]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"TT", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnTT.csv")
!cat test_ZnnTT.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.15192434117623954,0.16160600124525362,0.05990903799618909
SR_med_Znn_0j,0.26283716021924386,0.27867629989587456,0.05683705317800215
SR_high_Znn,0.19666149666972274,0.2093053414823378,0.06040861032532223


In [37]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"VV", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnVV.csv")
!cat test_ZnnVV.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.1483714893452771,0.16160600124525362,0.08189369081592329
SR_med_Znn_0j,0.26013433304802114,0.27867629989587456,0.0665358584665488
SR_high_Znn,0.19536985064186452,0.2093053414823378,0.06657971909259285


In [38]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"VVLF", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnVVLF.csv")
!cat test_ZnnVVLF.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.1481565936741278,0.16160600124525362,0.08322344137898047
SR_med_Znn_0j,0.25936547149474093,0.27867629989587456,0.06929483565107253
SR_high_Znn,0.19409831837397656,0.2093053414823378,0.07265473016915082


In [39]:
test_Znn = calculate("vhbb_Znn-2017.root", "Znn", 0,"j", 15, 0, 1)
test_Znn['ratio=(AS_0-AS)/AS_0'] = (test_Znn['AsimovSignificance_0'] - test_Znn['AsimovSignificance'])/test_Znn['AsimovSignificance_0']
#print(test_Znn)
test_Znn.to_csv("test_ZnnVj.csv")
!cat test_ZnnVj.csv

,AsimovSignificance,AsimovSignificance_0,ratio=(AS_0-AS)/AS_0
SR_med_Znn_ge1j,0.14991192333466702,0.16160600124525362,0.07236165625334448
SR_med_Znn_0j,0.26316431442608257,0.27867629989587456,0.055663095410653625
SR_high_Znn,0.19671635310828592,0.2093053414823378,0.06014652222869424
